# EX2 - Community detection
## In this exercise  you will use the famous "Zachary’s Karate club" network.
### By the way - Amir Rubin is a member of the "Karate club"-club :
### https://en.wikipedia.org/wiki/Zachary%27s_karate_club#Zachary_Karate_Club_Club

# Get packages needed for this task

In [ ]:
!pip install networkx==2.2
import networkx as nx
print(nx.__version__)

# Get Data

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import community

G = nx.karate_club_graph()
print("Community membership:")

for n in G.nodes():
    print(str(n) +": "+ str(G.node[n]['club']))


## Lets generate a nice coloring schema, where the main nodes are in different colors, and each community has it own color

In [ ]:
real_comms_colors = [2] + [0 if ('Hi' in G.node[n]['club']) else 1 for n in range(1,G.number_of_nodes()-1)] + [2]

# Visualize (dummy)

In [ ]:
nx.draw_circular(G, with_labels=True,node_color= real_comms_colors)

## The above is just a random sorting on a circle. We can do better than this!
### Experiment with the three options below

In [ ]:
#nx.draw(G, pos=nx.random_layout(G), with_labels=True, node_color  = real_comms_colors)
#nx.draw(G, pos=nx.spectral_layout(G), with_labels=True, node_color  = real_comms_colors)
nx.draw_networkx(G, pos=nx.spring_layout(G), with_labels=True,node_color  = real_comms_colors)
plt.show()

# Community Detection

## Girvan Newman

In [ ]:
communities_generator = community.girvan_newman(G)
i=0
while (True):
    i+=1
    print("Level " + str(i))
    print(next(communities_generator))

# Q: What is each level? Select one you think is the best and output it.

In [ ]:
best_level = #TODO - Select one you think works well (hint: in the original graph we had only two communities...)

In [ ]:
communities_generator = community.girvan_newman(G)
i=0
while True:
    i+=1
    comms = next(communities_generator)
    if i == best_level:
        break
part_i=0
for part in comms:
    print("")
    print("Community number " +str(part_i))        
    for node in part:
        print(str(node) + ": " + str(G.node[node]))
    part_i+=1

# Q: Is this a good partition? If you have nodes misplaced, why?

# A:

# Greedy modularity
## Lets use Louvain algo to find communities

In [ ]:
partition_by_Algo = community.greedy_modularity_communities(G)

partition = dict()
part_i=0
for part in partition_by_Algo:
    print("")
    print("Community number " +str(part_i))        
    for node in part:
        print(str(node) + ": " + str(G.node[node]))
        partition[node] =  part_i        
    part_i+=1


# Q: Two of the nodes is assigned to the "wrong" community.
## which nodes? 
## Why did this happend (look at the neighbours)?


# A:

# Plot using the Louvain partition
# Small task:
## build a list of colors based on the partition to be used to color the graph

In [ ]:
colors = #TODO

In [ ]:
nx.draw_networkx(G, pos=nx.kamada_kawai_layout(G), with_labels=True, node_color  = colors)
plt.show()

# Plot based on the communities detected

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=2.)

    pos_nodes = _position_nodes(g, partition, scale=.8)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = np.array(pos_communities[node] + pos_nodes[node])

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

def test():
    pos = community_layout(G, partition)
    
    colors=[]
    for i in range(len(G.nodes())):
        colors.append(partition[i]*1.0)
    print()
    
    labels = dict()
    for i in G.nodes():
        labels[i] = G.nodes[i]['club']
    
    nx.draw(G, pos, node_color=colors, with_labels=True, labels=labels, font_size=8)
    plt.show()
    return
test()
partition.values()

# Q: is it easier to analyze the network this way? What if the community detection algorithm gave bad results? 

# A: